In [ ]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
import random

# Set random seeds to ensure deterministic behavior
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# Constants
NUM_CHANNELS = 12  # Number of EEG channels
SEGMENT_LENGTH = 1200
MODEL_PATH = r"/content/final_eegnet_model.keras"
LABEL_ENCODER_PATH = r"/content/label_encoder_final.pkl"

# --------------------------------------
# Step 1: Load CSV Data
# --------------------------------------
csv_file = "/content/segmented_letter_data.csv"  # Replace with your actual CSV file path
df = pd.read_csv(csv_file)

# Remove non-numeric columns (label and participant)
df = df.select_dtypes(include=[np.number])

# Trim extra columns if not divisible by segment_length * channels
num_columns = df.shape[1]
expected_features = SEGMENT_LENGTH * NUM_CHANNELS
columns_to_trim = num_columns % expected_features

if columns_to_trim != 0:
    df = df.iloc[:, :-columns_to_trim]  # Trim the extra columns

print(f"✅ Shape after trimming: {df.shape}")

print("\n🔚 Last few columns of the DataFrame:")
print(df.iloc[:, -10:].head())  # Show the last 10 columns for the first few rows
print(f"🔢 Last column indices: {df.columns[-10:].tolist()}")  # Print their names/indexes


# Reshape: (samples, segment_length, channels)
X_temp = df.values.reshape(-1, SEGMENT_LENGTH, NUM_CHANNELS)

# Transpose to match model's expected shape: (samples, channels, time_steps, 1)
X_new = np.transpose(X_temp, (0, 2, 1))      # Now (samples, 12, 1200)
X_new = X_new[..., np.newaxis]              # Now (samples, 12, 1200, 1)

print(f"✅ Final input shape to model: {X_new.shape}")

# --------------------------------------
# Step 2: Load Model and Label Encoder
# --------------------------------------
model = load_model(MODEL_PATH)
model.trainable = False

with open(LABEL_ENCODER_PATH, "rb") as f:
    label_encoder = pickle.load(f)

# --------------------------------------
# Step 3: Predict
# --------------------------------------
# Optional: Check for NaN or Inf values
print(f"🧼 NaNs: {np.isnan(X_new).sum()}, Infs: {np.isinf(X_new).sum()}")
X_new = np.nan_to_num(X_new)

# Run prediction
y_pred_probs = model.predict(X_new)
print("🧠 Prediction probabilities for first sample:", y_pred_probs[0])

# Decode predicted classes
y_pred_labels = np.argmax(y_pred_probs, axis=-1)
predicted_letters = label_encoder.inverse_transform(y_pred_labels)

# Display predictions
print("\n🔤 Predicted Letters:")
for i, letter in enumerate(predicted_letters):
    print(f"  Sample {i+1}: {letter}")


✅ Shape after trimming: (1, 14400)

🔚 Last few columns of the DataFrame:
   feature_14390  feature_14391  feature_14392  feature_14393  feature_14394  \
0       1.492517       1.049635       0.005963      -0.036129       0.865046   

   feature_14395  feature_14396  feature_14397  feature_14398  feature_14399  
0       0.919642      -0.263105       1.116019      -0.525958       0.155263  
🔢 Last column indices: ['feature_14390', 'feature_14391', 'feature_14392', 'feature_14393', 'feature_14394', 'feature_14395', 'feature_14396', 'feature_14397', 'feature_14398', 'feature_14399']
✅ Final input shape to model: (1, 12, 1200, 1)
🧼 NaNs: 0, Infs: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
🧠 Prediction probabilities for first sample: [9.59261670e-04 7.55274415e-01 2.81864312e-03 3.79343075e-03
 3.07595101e-03 2.48910603e-03 6.36705104e-03 2.36299615e-02
 3.07933486e-04 9.30671860e-03 4.10350971e-03 1.31583195e-02
 3.34845227e-03 2.78735976e-03 5.47834486e-03 4.25632671e-03
 2.98235267e-02 3.91069